<p style="font-size: 1.1rem; color:#9beb3b; text-align: center; font-family: monospace ">
    <strong>
       Importando bibliotecas
    </strong>
</p>

In [30]:
import pandas as pd
from openpyxl import Workbook, load_workbook
from openpyxl.styles import numbers
import xlwings as xw


<p style="font-size: 1.1rem; color:#9beb3b; text-align: center; font-family: monospace ">
    <strong><br/><br/>
       Criando planilha de controle de entregas
    </strong>
</p>

In [31]:
controle_path = 'dados/contorle_entregas.xlsx'

try: 
    wb = Workbook()

    ws = wb.active

    ws.title = 'Entregas'


    ws.append([
    'Data',
    'Estado',
    'Motorista',
    'Entregas Previstas',
    'Entregas no Prazo',
    'Entregas Realizadas'
    ])

    ws.append(['01/01/2026','RJ','Léo','10','6','7'])
    ws.append(['02/01/2026','SP','Rafael','20','5','0'])
    ws.append(['02/01/2026','MG','Maria','5','2','3'])
    ws.append(['03/01/2026','MG','Arthur','15','2','1'])
    ws.append(['03/01/2026','SP','Vinicius','50','1','5'])
    ws.append(['01/01/2026','ES','Marcia','36','11','3'])

    wb.save(controle_path)
    print('Planilha')

except Exception as e:
    print(e)



Planilha


<p style="font-size: 1.1rem; color:#9beb3b; text-align: center; font-family: monospace ">
    <strong><br/><br/>
       Inserindo novos indicadores
    </strong>
</p>

In [32]:
wb = load_workbook(controle_path)
ws = wb.active

#Criando novas colunas
ws['G1'] = "Taxa de Sucesso"
ws['H1'] = "Taxa de Atraso"

for linha in range(2, ws.max_row+1):
    ws[f'G{linha}'] = f'=F{linha}/D{linha}'
    ws[f'H{linha}'] = f'=1-G{linha}'

wb.save(controle_path)
print("Formulas inseridas com sucesso!")
 

Formulas inseridas com sucesso!


<p style="font-size: 1.1rem; color:#9beb3b; text-align: center; font-family: monospace ">
    <strong><br/><br/>
       Formatando as taxas como percentual
    </strong>
</p>

In [33]:
wb = load_workbook(controle_path)
ws = wb.active

for linha in range(2, ws.max_row+1):
    ws[f'G{linha}'].number_format = '0.00%' 
    ws[f'H{linha}'].number_format = '0.00%'

wb.save(controle_path)
print("Tipagem alterado com sucesso")



Tipagem alterado com sucesso


<p style="font-size: 1.1rem; color:#9beb3b; text-align: center; font-family: monospace ">
    <strong><br/><br/>
        Resumo gerencial por estado
    </strong>
</p>

In [41]:
df = pd.read_excel(controle_path,sheet_name='Entregas')

df['Taxa de sucesso 2'] = df['Entregas Realizadas'] / df['Entregas Previstas'] 


# resumo_estado = df.groupby('Estado').agg(
#     Total_Prevista = ('Entregas Previstas','sum'),
#     Total_entregas_realizadas = ('Entregas Realizadas','sum'),
#     Taxa_sucesso = ('Taxa de Sucesso', 'mean'),
#     Taxa_atraso = ('Taxa de Atraso','mean')
# )

resumo_estado = df.groupby('Estado').agg({
    'Entregas Previstas' :'sum',
    'Entregas Realizadas': 'sum',
    'Taxa de Sucesso': 'mean',
    'Taxa de Atraso': 'mean'
})

wb = load_workbook(controle_path)

if  'Resumo' in wb.sheetnames:
    del wb['Resumo']

ws_resumo = wb.create_sheet('Resumo')

ws_resumo.append([
    'Estado',
    'Entregas Previstas',
    'Entregas Realizadas',
    'Taxa de Sucesso',
    'Taxa de Atraso'
     ])



for estado,valores in resumo_estado.iterrows():
    ws_resumo.append([
        estado,
        valores['Entregas Previstas'],
        valores['Entregas Realizadas'],
        valores['Taxa de Sucesso'],
        valores['Taxa de Atraso']
    ])


for linha in range(2, ws_resumo.max_row +1):
     ws_resumo[f'D{linha}'].number_format= '0.00%'    
     ws_resumo[f'E{linha}'].number_format= '0.00%'

wb.save(controle_path)






<p style="font-size: 1.1rem; color:#9beb3b; text-align: center; font-family: monospace ">
    <strong><br/><br/>
        Cálculos de taxas e inserindo no excel
    </strong>
</p>